In [ ]:


import pandas as pd

train_data = pd.read_csv("ai4scup-cns_v3/mol_train.csv")
train_data.columns=["SMILES", "TARGET"]
train_data.to_csv("./ai4scup-cns_v3/mol_train.csv", index=False)
test_data = pd.read_csv("./ai4scup-cns_v3/mol_test.csv")
test_data.columns=["SMILES", "TARGET"]
test_data.to_csv("./ai4scup-cns_v3/mol_test.csv", index=False)

print("------ train data ------")
print(train_data)
print("POSITIVE:", sum(train_data["TARGET"]==1))
print("NEGETIVE:", sum(train_data["TARGET"]==0))

print("------ test  data ------")
print(test_data)
print("POSITIVE:", sum(test_data["TARGET"]==1))
print("NEGETIVE:", sum(test_data["TARGET"]==0))

In [ ]:


from rdkit import Chem
from rdkit.Chem import Descriptors

def calculate_1dqsar_repr(smiles):
    # 从 SMILES 字符串创建分子对象
    mol = Chem.MolFromSmiles(smiles)
    # 计算分子的分子量
    mol_weight = Descriptors.MolWt(mol)
    # 计算分子的 LogP 值
    log_p = Descriptors.MolLogP(mol)
    # 计算分子中的氢键供体数量
    num_h_donors = Descriptors.NumHDonors(mol)
    # 计算分子中的氢键受体数量
    num_h_acceptors = Descriptors.NumHAcceptors(mol)
    # 计算分子的表面积极性
    tpsa = Descriptors.TPSA(mol)
    # 计算分子中的可旋转键数量
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    # 计算分子中的芳香环数量
    num_aromatic_rings = Descriptors.NumAromaticRings(mol)
    # 计算分子中的脂环数量
    num_aliphatic_rings = Descriptors.NumAliphaticRings(mol)
    # 计算分子中的饱和环数量
    num_saturated_rings = Descriptors.NumSaturatedRings(mol)
    # 计算分子中的杂原子数量
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    # 计算分子中的价电子数量
    num_valence_electrons = Descriptors.NumValenceElectrons(mol)
    # 计算分子中的自由基电子数量
    num_radical_electrons = Descriptors.NumRadicalElectrons(mol)
    # 计算分子的 QED 值
    qed = Descriptors.qed(mol)
    # 返回所有计算出的属性值
    return [mol_weight, log_p, num_h_donors, num_h_acceptors, tpsa, num_rotatable_bonds, num_aromatic_rings,
            num_aliphatic_rings, num_saturated_rings, num_heteroatoms, num_valence_electrons, num_radical_electrons,qed]


train_data["1dqsar_mr"] = train_data["SMILES"].apply(calculate_1dqsar_repr)
test_data["1dqsar_mr"] = test_data["SMILES"].apply(calculate_1dqsar_repr)

In [ ]:


print(train_data["1dqsar_mr"][0])
print("一维的特征有%d个"%len(train_data["1dqsar_mr"][0]))

In [ ]:


import numpy as np
from rdkit.Chem import AllChem

def calculate_2dqsar_repr(smiles):    
    # 将smiles字符串转换为rdkit的分子对象
    mol = Chem.MolFromSmiles(smiles)
    # 计算Morgan指纹（半径为3，长度为1024位）
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=1024)
    # 返回numpy数组格式的指纹
    return np.array(fp)

train_data["2dqsar_mr"] = train_data["SMILES"].apply(calculate_2dqsar_repr) 
test_data["2dqsar_mr"] = test_data["SMILES"].apply(calculate_2dqsar_repr) 

In [ ]:


print(train_data["2dqsar_mr"][0].tolist())
print("二维的特征有%d个"%len(train_data["2dqsar_mr"][0]))

In [ ]:


from rdkit.Chem import rdPartialCharges
from sklearn.preprocessing import StandardScaler

def calculate_3dqsar_repr(SMILES, max_atoms=100, three_d=False):
    mol = Chem.MolFromSmiles(SMILES)  # 从SMILES表示创建分子对象
    mol = Chem.AddHs(mol)  # 添加氢原子
    if three_d:
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # 计算3D坐标
    else:
        AllChem.Compute2DCoords(mol)  # 计算2D坐标
    natoms = mol.GetNumAtoms()  # 获取原子数量
    rdPartialCharges.ComputeGasteigerCharges(mol)  # 计算分子的Gasteiger电荷
    charges = np.array([float(atom.GetProp("_GasteigerCharge")) for atom in mol.GetAtoms()])  # 获取电荷值
    coords = mol.GetConformer().GetPositions()  # 获取原子坐标
    coulomb_matrix = np.zeros((max_atoms, max_atoms))  # 初始化库仑矩阵
    n = min(max_atoms, natoms)
    for i in range(n):  # 遍历原子
        for j in range(i, n):
            if i == j:
                coulomb_matrix[i, j] = 0.5 * charges[i] ** 2
            if i != j:
                delta = np.linalg.norm(coords[i] - coords[j])  # 计算原子间距离
                if delta != 0:
                    coulomb_matrix[i, j] = charges[i] * charges[j] / delta  # 计算库仑矩阵的元素值
                    coulomb_matrix[j, i] = coulomb_matrix[i, j]
    coulomb_matrix = np.where(np.isinf(coulomb_matrix), 0, coulomb_matrix)  # 处理无穷大值
    coulomb_matrix = np.where(np.isnan(coulomb_matrix), 0, coulomb_matrix)  # 处理NaN值
    return coulomb_matrix.reshape(max_atoms*max_atoms).tolist()  # 将库仑矩阵转换为列表并返回


train_data["3dqsar_mr"] = train_data["SMILES"].apply(calculate_3dqsar_repr) 
test_data["3dqsar_mr"] = test_data["SMILES"].apply(calculate_3dqsar_repr) 

In [ ]:


print("三维的特征有%d个"%len(train_data["3dqsar_mr"][0]))
print(train_data["3dqsar_mr"][0])

In [ ]:


train_data


# ### 下面是特征融合，将一维、二维、三维进行了特征融合

In [ ]:


# 将这些列合并为特征矩阵X,处理train_data
# 使用values属性可以直接获得每列中的数据，返回结果为numpy数组
X_1dqsar = np.stack(train_data["1dqsar_mr"].values)
X_2dqsar = np.stack(train_data["2dqsar_mr"].values)
X_3dqsar = np.stack(train_data["3dqsar_mr"].values)
# 水平堆叠这些特征向量，形成一个更宽的特征矩阵
X = np.hstack((X_1dqsar, X_2dqsar, X_3dqsar))
# 对特征进行标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X)
print("X_train shape:" ,X_train.shape)

y_train = np.array(train_data["TARGET"].values.tolist())
print("y_train shape:", y_train.shape)

# 处理test_data
# 获取特征向量
X_1dqsar_test = np.stack(test_data["1dqsar_mr"].values)
X_2dqsar_test = np.stack(test_data["2dqsar_mr"].values)
X_3dqsar_test = np.stack(test_data["3dqsar_mr"].values)

# 将特征向量合并为特征矩阵X_test，与训练数据相对应
X_test = np.hstack((X_1dqsar_test, X_2dqsar_test, X_3dqsar_test))
# 使用相同的scaler可以确保测试数据的标准化方式与训练数据一致
X_test = scaler.transform(X_test)
print("X_test_scaled shape:" ,X_test.shape)
y_test = np.array(test_data["TARGET"].values.tolist())
print("y_test shape", y_test.shape)

In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score

# 特征选择: 使用随机森林重要性评分进行特征选择
select = SelectFromModel(RandomForestClassifier(n_estimators=100))
X_train_selected = select.fit_transform(X_train, y_train)

# 降维: 使用PCA进一步减少维度到合理数量
pca = PCA(n_components=0.85)  # 保留85%的方差
X_train_pca = pca.fit_transform(X_train_selected)

# 创建机器学习算法模型: 随机森林分类器
clf = RandomForestClassifier(n_estimators=800)

# # 训练模型
# clf.fit(X_train_pca, y_train)

# 特征选择和降维可以与模型训练结合成一个pipeline
pipeline = Pipeline([
    ('feature_selection', select),
    ('dimension_reduction', pca),
    ('classification', clf)
])

# 划分训练数据和测试数据
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# 训练pipeline
pipeline.fit(X_train_split, y_train_split)

# 在测试集上评估模型
y_pred = pipeline.predict(X_test_split)

# 计算 F2 分数
f2_score = fbeta_score(y_test_split, y_pred, beta=2)

print(f"F2 Score: {f2_score}")